In [18]:
library(ArchR)

In [19]:
addArchRThreads(threads = 48)

Setting default number of Parallel threads to 48.



In [20]:
addArchRGenome("hg19")

Setting default genome to Hg19.



In [21]:
# 设置文件路径和样本名称
file_path <- "/mnt/disk1/xiaojk/data/DLPFC/GSM5138548_RL2085_20y_snATAC_fragments.tsv.gz"
sample_name <- "GSM5138548_RL2085_20y_snATAC_fragments"

# 将文件路径和样本名称添加到相应的变量中
inputFiles <- c(file_path)
sampleNames <- c(sample_name)

# 调用createArrowFiles函数
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = sampleNames,
  filterTSS = 4,
  filterFrags = 1000,
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

filterFrags is no longer a valid input. Please use minFrags! Setting filterFrags value to minFrags!

filterTSS is no longer a valid input. Please use minTSS! Setting filterTSS value to minTSS!

Using GeneAnnotation set by addArchRGenome(Hg19)!

Using GeneAnnotation set by addArchRGenome(Hg19)!

ArchR logging to : ArchRLogs/ArchR-createArrows-12f8417e999bfb-Date-2023-09-19_Time-15-11-21.517342.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-09-19 15:11:21.661225 : Batch Execution w/ safelapply!, 0 mins elapsed.

(GSM5138548_RL2085_20y_snATAC_fragments : 1 of 1) Checking if completed file exists!

2023-09-19 15:11:21.754629 : (GSM5138548_RL2085_20y_snATAC_fragments : 1 of 1) Arrow Exists! Marking as completed since force = FALSE!, 0.002 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-12f8417e999bfb-Date-2023-09-19_Time-15-11-21.517342.log



In [22]:
ArrowFiles

[1] "GSM5138548_RL2085_20y_snATAC_fragments.arrow"

In [23]:
projDLPFC1 <- ArchRProject(
  ArrowFiles = ArrowFiles,
  outputDirectory = "/mnt/disk1/xiaojk/data/DLPFC/",
  copyArrows = TRUE #This is recommened so that if you modify the Arrow files you have an original copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Hg19)!

Using GeneAnnotation set by addArchRGenome(Hg19)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [24]:
projDLPFC1


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /mnt/disk1/xiaojk/data/DLPFC 
samples(1): GSM5138548_RL2085_20y_snATAC_fragments
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 6940
medianTSS(1): 10.4915
medianFrags(1): 9163

In [25]:
paste0("Memory Size = ", round(object.size(projDLPFC1) / 10^6, 3), " MB")
getAvailableMatrices(projDLPFC1)

[1] "Memory Size = 36.67 MB"

[1] "GeneScoreMatrix" "TileMatrix"

In [9]:
projDLPFC1 <- addIterativeLSI(
    ArchRProj = projDLPFC1,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-12f841422bd6f8-Date-2023-08-17_Time-18-41-50.504893.log
If there is an issue, please report to github with logFile!

2023-08-17 18:41:50.665772 : Computing Total Across All Features, 0.001 mins elapsed.

2023-08-17 18:41:52.380106 : Computing Top Features, 0.029 mins elapsed.

###########
2023-08-17 18:41:53.598807 : Running LSI (1 of 2) on Top Features, 0.049 mins elapsed.
###########

2023-08-17 18:41:53.613891 : Creating Partial Matrix, 0.05 mins elapsed.

2023-08-17 18:42:12.342488 : Computing LSI, 0.362 mins elapsed.

2023-08-17 18:43:02.109652 : Identifying Clusters, 1.191 mins elapsed.

2023-08-17 18:43:13.059183 : Identified 7 Clusters, 1.374 mins elapsed.

2023-08-17 18:43:13.075676 : Saving LSI Iteration, 1.374 mins elapsed.

2023-08-17 18:43:27.779097 : Creating Cluster Matrix on the total Group Features, 1.619 mins elapsed.

2023-08-17 18:43:35.566083 : Computing Variable Features, 1.749 mins elapsed.

#

In [10]:
projDLPFC1 <- addClusters(
    input = projDLPFC1,
    reducedDims = "IterativeLSI",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.8
)

ArchR logging to : ArchRLogs/ArchR-addClusters-12f84111350719-Date-2023-08-17_Time-18-44-40.571428.log
If there is an issue, please report to github with logFile!

2023-08-17 18:44:40.752608 : Running Seurats FindClusters (Stuart et al. Cell 2019), 0.001 mins elapsed.

Computing nearest neighbor graph

Computing SNN



Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 6940
Number of edges: 419631

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8769
Number of communities: 12
Elapsed time: 0 seconds


2023-08-17 18:44:49.526374 : Testing Biased Clusters, 0.148 mins elapsed.

2023-08-17 18:44:49.56233 : Testing Outlier Clusters, 0.148 mins elapsed.

2023-08-17 18:44:49.56497 : Assigning Cluster Names to 12 Clusters, 0.148 mins elapsed.

2023-08-17 18:44:49.632112 : Finished addClusters, 0.149 mins elapsed.



In [11]:
head(projDLPFC1$Clusters)
table(projDLPFC1$Clusters)

[1] "C11" "C8"  "C4"  "C5"  "C4"  "C5"


  C1  C10  C11  C12   C2   C3   C4   C5   C6   C7   C8   C9 
 798  857  196 1147  375  403  855  940   47  577  340  405 

In [12]:
projDLPFC1 <- addGroupCoverages(ArchRProj = projDLPFC1, groupBy = "Clusters")

ArchR logging to : ArchRLogs/ArchR-addGroupCoverages-12f841494226e6-Date-2023-08-17_Time-18-44-49.66574.log
If there is an issue, please report to github with logFile!

C1 (1 of 12) : CellGroups N = 2

C2 (2 of 12) : CellGroups N = 2

C3 (3 of 12) : CellGroups N = 2

C4 (4 of 12) : CellGroups N = 2

C5 (5 of 12) : CellGroups N = 2

C6 (6 of 12) : CellGroups N = 2

C7 (7 of 12) : CellGroups N = 2

C8 (8 of 12) : CellGroups N = 2

C9 (9 of 12) : CellGroups N = 2

C10 (10 of 12) : CellGroups N = 2

C11 (11 of 12) : CellGroups N = 2

C12 (12 of 12) : CellGroups N = 2

2023-08-17 18:44:50.715198 : Creating Coverage Files!, 0.017 mins elapsed.

2023-08-17 18:44:50.717178 : Batch Execution w/ safelapply!, 0.018 mins elapsed.

2023-08-17 18:45:36.059184 : Adding Kmer Bias to Coverage Files!, 0.773 mins elapsed.

Completed Kmer Bias Calculation

Adding Kmer Bias (1 of 24)

Adding Kmer Bias (2 of 24)

Adding Kmer Bias (3 of 24)

Adding Kmer Bias (4 of 24)

Adding Kmer Bias (5 of 24)

Adding Kmer

In [13]:
pathToMacs2 <- "/mnt/disk1/xiaojk/anaconda/envs/ArchR/bin/macs2"
projDLPFC1 <- addReproduciblePeakSet(
    ArchRProj = projDLPFC1, 
    groupBy = "Clusters", 
    pathToMacs2 = pathToMacs2
)

ArchR logging to : ArchRLogs/ArchR-addReproduciblePeakSet-12f8413ebd7624-Date-2023-08-17_Time-18-46-22.228563.log
If there is an issue, please report to github with logFile!

Calling Peaks with Macs2

2023-08-17 18:46:22.385656 : Peak Calling Parameters!, 0.003 mins elapsed.



    Group nCells nCellsUsed nReplicates nMin nMax maxPeaks
C1     C1    798        540           2   40  500   150000
C2     C2    375        375           2   40  335   150000
C3     C3    403        403           2   40  363   150000
C4     C4    855        540           2   40  500   150000
C5     C5    940        540           2   40  500   150000
C6     C6     47         41           2   23   26    20500
C7     C7    577        540           2   40  500   150000
C8     C8    340        340           2   40  300   150000
C9     C9    405        405           2   40  365   150000
C10   C10    857        540           2   40  500   150000
C11   C11    196        196           2   40  156    98000
C12   C12   1147        540           2   40  500   150000


2023-08-17 18:46:22.394079 : Batching Peak Calls!, 0.003 mins elapsed.

2023-08-17 18:46:22.40161 : Batch Execution w/ safelapply!, 0 mins elapsed.

2023-08-17 18:49:02.980287 : Identifying Reproducible Peaks!, 2.679 mins elapsed.

2023-08-17 18:49:09.846745 : Creating Union Peak Set!, 2.794 mins elapsed.

Converged after 5 iterations!

Plotting Ggplot!

2023-08-17 18:49:18.114217 : Finished Creating Union Peak Set (228269)!, 2.931 mins elapsed.



In [14]:
peakSet <- getPeakSet(projDLPFC1)
peakSet

GRanges object with 228269 ranges and 13 metadata columns:
      seqnames              ranges strand |     score replicateScoreQuantile
         <Rle>           <IRanges>  <Rle> | <numeric>              <numeric>
   C1     chr1       752484-752984      * |   4.84273                  0.839
   C7     chr1       753197-753697      * |   5.54167                  0.580
  C10     chr1       754100-754600      * |   2.48002                  0.559
   C4     chr1       762701-763201      * |  10.37470                  0.964
   C6     chr1       773670-774170      * |   2.59230                  0.601
  ...      ...                 ...    ... .       ...                    ...
   C3     chrX 154849018-154849518      * |   2.89553                  0.349
  C10     chrX 154902734-154903234      * |   2.48002                  0.559
   C7     chrX 154948551-154949051      * |   4.53534                  0.681
  C10     chrX 154997020-154997520      * |   4.27735                  0.880
   C4     chrX 15

In [26]:
getAvailableMatrices(projDLPFC1)

[1] "GeneScoreMatrix" "TileMatrix"

In [27]:
seqnames_ranges <- paste(seqnames_info, ranges_info, sep = "-")

ERROR: Error in eval(expr, envir, enclos): object 'seqnames_info' not found


In [28]:
GeneScoreMatrix <- getMatrixFromProject(
  ArchRProj = projDLPFC1,
  useMatrix = "GeneScoreMatrix",
  useSeqnames = NULL,
  verbose = TRUE,
  binarize = FALSE,
  threads = getArchRThreads(),
  logFile = createLogFile("getMatrixFromProject")
)

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-12f8413479ffda-Date-2023-09-19_Time-15-11-54.303383.log
If there is an issue, please report to github with logFile!

2023-09-19 15:12:29.279223 : Organizing colData, 0.583 mins elapsed.

2023-09-19 15:12:29.281608 : Organizing rowData, 0.583 mins elapsed.

2023-09-19 15:12:29.284001 : Organizing rowRanges, 0.583 mins elapsed.

2023-09-19 15:12:29.289242 : Organizing Assays (1 of 1), 0.583 mins elapsed.

2023-09-19 15:12:29.294122 : Constructing SummarizedExperiment, 0.583 mins elapsed.

2023-09-19 15:12:30.247652 : Finished Matrix Creation, 0.599 mins elapsed.



In [29]:
seqnames_info <- rowData(GeneScoreMatrix)$seqnames
start_info <- rowData(GeneScoreMatrix)$start
end_info <- rowData(GeneScoreMatrix)$end
seqnames_ranges <- paste(seqnames_info, start_info, end_info,sep = "-")
rownames(GeneScoreMatrix) <- seqnames_ranges
GeneScoreMatrix

class: SummarizedExperiment 
dim: 23127 6940 
metadata(0):
assays(1): GeneScoreMatrix
rownames(23127): chr1-11874-14409 chr1-29961-14362 ...
  chrX-154563986-154505500 chrX-154842622-154718673
rowData names(6): seqnames start ... name idx
colnames(6940):
  GSM5138548_RL2085_20y_snATAC_fragments#GAAGTCTCATGCACTA-1
  GSM5138548_RL2085_20y_snATAC_fragments#GTTGGTATCCCGCAAG-1 ...
  GSM5138548_RL2085_20y_snATAC_fragments#GTAGTACAGGCGCTTC-1
  GSM5138548_RL2085_20y_snATAC_fragments#AGTCCGGAGTGAGCTG-1
colData names(13): BlacklistRatio nDiFrags ... Sample TSSEnrichment

In [30]:
# 获取当前的列名
current_colnames <- colnames(GeneScoreMatrix)
# 使用正则表达式将列名中的前缀部分替换为空字符串
new_colnames <- gsub("GSM5138548_", "", current_colnames)
new_colnames <- gsub("_20y_snATAC_fragments#", "#", new_colnames)

# 更新 RangedSummarizedExperiment 对象的列名
colnames(GeneScoreMatrix) <- new_colnames

In [31]:
# 获取数据矩阵
data_matrix <- assay(GeneScoreMatrix)
# 将数据矩阵转换为稀疏矩阵
sparse_matrix <- as(data_matrix, "sparseMatrix")
# 矩阵转置

In [32]:
seurat_obj <- CreateSeuratObject(counts = sparse_matrix)
obs_names <- colnames(seurat_obj)

Warning message:
“Non-unique features (rownames) present in the input matrix, making unique”


In [33]:
var_names <- rownames(seurat_obj)

In [34]:
rna_assay <- seurat_obj[["RNA"]]

# 查看 "RNA" assay 的值（前100个行和列）
# print(rna_assay[1:100, 1:100])


In [35]:
#根据元数据对碱基序列进行细胞类型注释
meta <- read.delim("/mnt/disk1/xiaojk/data/DLPFC/meta.tsv", header = TRUE)
obs_cell_types <- meta$cell_type[match(obs_names, meta$cellId)]
cell_type_counts <- table(obs_cell_types)
sum(cell_type_counts)

[1] 5262

In [ ]:
# 获取匹配到的观测对象索引
filtered_indices <- !is.na(obs_cell_types)
# 使用 filtered_indices 子集化 Seurat 对象
seurat_obj_filtered <- seurat_obj[,filtered_indices]
seurat_obj_filtered$cell_type <- obs_cell_types[filtered_indices]
seurat_obj_filtered

In [ ]:
#删除不必要的特征
seurat_obj_filtered$orig.ident <- NULL
seurat_obj_filtered$nCount_RNA <- NULL
seurat_obj_filtered$nFeature_RNA <- NULL

In [ ]:
library(SeuratDisk)

SaveH5Seurat(seurat_obj_filtered, filename = "/mnt/disk1/xiaojk/data/DLPFC/seurat_data.h5Seurat")
Convert("/mnt/disk1/xiaojk/data/DLPFC/seurat_data.h5Seurat",dest = "h5ad")